# metacognitive prompting

### import statements

In [1]:
import uuid

from openai import OpenAI
from dotenv import load_dotenv
from ipywidgets import widgets, Output
from IPython.display import display
import threading
import os

from models.user import User
from models.chat_message import ChatMessage
from models.conversation import Conversation

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%run ../services/db.py

In [4]:
from services.db import mongo_client, ping_client, set_db, set_collection
db_client = mongo_client()
ping_client(db_client)

MongoClient(host=['ac-yqldgbe-shard-00-02.vfkvijy.mongodb.net:27017', 'ac-yqldgbe-shard-00-01.vfkvijy.mongodb.net:27017', 'ac-yqldgbe-shard-00-00.vfkvijy.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster', tlscafile='../isrgrootx1.pem', authsource='admin', replicaset='atlas-gj1ls4-shard-0', tls=True, server_api=<pymongo.server_api.ServerApi object at 0x1084c8c50>, uuidrepresentation=4)
Pinged your deployment. You successfully connected to MongoDB!


In [5]:
load_dotenv()
text_input=""
chat_output=Output()
active = True
processing_message = threading.Event()

In [6]:
test_user = User(email="piet.varanvenzwijn@test.com", password="secret")
print(f"User class instance:\n{test_user.__dict__}")
saved_test_user_id = test_user.save(db_client)
print(f"Saved test user dictionary:\n{saved_test_user_id}")

User class instance:
{'uid': UUID('88e83be0-39c0-45df-813f-ba4a0e24726c'), 'created_at': datetime.datetime(2024, 2, 27, 16, 32, 32, 769122), 'email': 'piet.varanvenzwijn@test.com', 'password': b'$2b$12$ZRAySAaJbm/WR7HhNbnZ6.QXhhZJ8T2RU66IfY6rAADieOU6khoFK'}
InsertOneResult(ObjectId('65de009198d3fa5616abc8d2'), acknowledged=True)
Saved test user dictionary:
65de009198d3fa5616abc8d2


In [11]:
test_conversation = Conversation(user_uid='88e83be0-39c0-45df-813f-ba4a0e24726c')
print(f"Conversation class instance:\n{test_conversation.__dict__}")
saved_test_conversation_id = test_conversation.save(db_client)
print(f"Saved test conversation ObjectId:\n{saved_test_conversation_id}")

Conversation class instance:
{'created_at': datetime.datetime(2024, 2, 27, 17, 20, 15, 640418), 'user_uid': UUID('88e83be0-39c0-45df-813f-ba4a0e24726c'), 'user_message_count': 0, 'user_login_count': 0, 'conversation_topics': ['New conversation topic']}
InsertOneResult(ObjectId('65de0bbf98d3fa5616abc8d3'), acknowledged=True)
Saved test conversation ObjectId:
65de0bbf98d3fa5616abc8d3


In [14]:
test_message= ChatMessage(author="piet van ravenzwijn", content="test", conversation_id=saved_test_conversation_id)
print(f"ChatMessage class instance:\n{test_message.__dict__}")
saved_test_message_id = test_message.save(db_client)
print(f"Saved test message ObjectId:\n{saved_test_message_id }")

ChatMessage class instance:
{'created_at': datetime.datetime(2024, 2, 27, 17, 59, 55, 316691), 'author': 'piet van ravenzwijn', 'content': 'test', 'conversation_id': ObjectId('65de0bbf98d3fa5616abc8d3')}
InsertOneResult(ObjectId('65de150b98d3fa5616abc8d4'), acknowledged=True)
Saved test message ObjectId:
65de150b98d3fa5616abc8d4


In [ ]:
apikey = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(
    api_key=apikey
)
print(openai_client)

:::{admonition} `stream_chatbot` docs
:class: dropdown 

### `stream_chatbot(message)`

This function interacts with the chatbot using stream functionality.

**Parameters:**
- `message`: The message sent by the user to the chatbot.

**Returns:**
- None

**Functionality:**
- Sends the user message to the chatbot.
- Receives and prints the response from the chatbot.
```

In [ ]:
def stream_chatbot(message):
    global chat_output
    chat_completion_stream = openai_client.chat.completions.create(
        messages=[
            {
                "role":"user",
                "content":message,
             },
        ],
        model="gpt-3.5-turbo",
        stream=True
    )
    print(chat_completion_stream)
    for chunk in chat_completion_stream:
        if chunk.choices[0].delta.content is not None:
            chat_output.append_stdout(f"{chunk.choices[0].delta.content}")
    chat_output.append_stdout(f"\n")

:::{admonition} `prompt_user` docs
:class: dropdown

### `prompt_user()`

This function prompts the user for input.

**Parameters:**
- None

**Returns:**
- `user_input`: The input provided by the user.

**Functionality:**
- Displays a prompt for the user to input a message.
- Returns the user's input.

In [ ]:
def prompt_user():
    user_input = input("You: ")
    return user_input

:::{admonition} `chatbot` docs
:class: dropdown

### `chatbot(user_message)`

This function interacts with the chatbot based on user input.

**Parameters:**
- `user_message`: The message provided by the user.

**Returns:**
- None

**Functionality:**
- Initiates interaction with the chatbot by passing the user's message.
- Prints the chatbots response.


In [ ]:
def chatbot(user_message):
    global active
    global chat_output
    processing_message.set()
    chat_output.append_stdout("Chatbot: ")
    if user_message.lower() in ["bye!", "quit", "exit"]:
        chat_output.append_stdout("BYE\n")
        active = False
    else:
        stream_chatbot(user_message)
    processing_message.clear()

:::{admonition} `print_user_message` docs
:class: dropdown

### `print_user_message(user_message)`

This function prints the user's message.

**Parameters:**
- `user_message`: The message provided by the user.

**Returns:**
- None

**Functionality:**
- Prints the user's message in the format: "You: [user_message]".


In [ ]:
def print_user_message(user_message):
    global chat_output
    chat_output.append_stdout(f"You: {user_message}\n")

In [ ]:
def handle_user_message(user_message):
    print_user_message(user_message)
    chatbot(user_message)

In [ ]:
def on_submit_button_clicked(b):
    if not processing_message.is_set():
        handle_user_message(text_input.value)
        text_input.value=""

In [ ]:
def chat_interface():
    global text_input
    text_input = widgets.Text(description="Your message: ")
    submit_button = widgets.Button(description="Submit", disabled=False)
    
    submit_button.on_click(on_submit_button_clicked)    
    
    display(chat_output, text_input, submit_button)

In [ ]:
def main():
    new_chat()
    chat_interface()

if __name__ == "__main__": main()